In [1]:
%load_ext autoreload
%autoreload 2
from Methods import sentenceSimilarityMethods,JobOffers,dataframeHelpers

In [2]:
import os
import nltk
import re
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
from scipy import spatial

In [4]:
autoreload JobOffers,dataframeHelpers

In [43]:
# Only run this if you want to redo the translations
JobOffers.writeTranslatedJobs(200)

In [44]:
rawOffers = JobOffers.readJsonFile("Data/output_v1.1.json")

In [45]:
translatedOffers = JobOffers.readJsonFile("Data/output_v1.1.translate.json")

In [46]:
jobsDf = dataframeHelpers.buildJobsDF()

In [47]:
jobsDf

,JOB_ID,ISCO,JOB_CONTENT,JOB_SCORES
0,12220025,21420,[We are a successful and independent engineeri...,"((tf.Tensor(-0.04822776, shape=(), dtype=float..."
1,10814567,21420,[We are a prestigious and successful engineeri...,"((tf.Tensor(-0.020631889, shape=(), dtype=floa..."
2,12401674,21420,[Bill Weyermann Partner AG has been producing ...,"((tf.Tensor(0.055472188, shape=(), dtype=float..."
3,12473085,62100,[Administration Communale de Valderuzforstarti...,"((tf.Tensor(0.019481275, shape=(), dtype=float..."
4,12905109,26220;24210,[The Straumann Group is experiencing internati...,"((tf.Tensor(-0.022287764, shape=(), dtype=floa..."
...,...,...,...,...
195,13761500,24210,[SA of the Horlogerie Manufacture Audemars Pig...,"((tf.Tensor(0.012489217, shape=(), dtype=float..."
196,13760809,21420,"[20, 01, 2017Henauer Gugler AGbern80 to 100 ...","((tf.Tensor(-0.026219312, shape=(), dtype=floa..."
197,14379272,24210,[Your tasks processing deviations conducting i...,"((tf.Tensor(0.0007885815, shape=(), dtype=floa..."
198,14384238,21620,[Text building block header_1 Jobbörse Text mo...,"((tf.Tensor(0.013943924, shape=(), dtype=float..."


In [48]:
jobsDf.to_csv("Data/AnnotatedJobs.csv")

In [95]:
annotatedJobs200 = pd.read_csv("Data/AnnotatedJobs200.csv", sep=";")
annotatedJobs200

,Unnamed: 0,JOB_ID,ISCO,JOB_CONTENT,JOB_SCORES,GREEN_IDENTITY
0,0,12220025,21420,['We are a successful and independent engineer...,tf.Tensor(\n[[-0.04822776 -0.02858711 -0.06559...,1
1,1,10814567,21420,['We are a prestigious and successful engineer...,tf.Tensor(\n[[-0.02063189 0.00667052 -0.01903...,1
2,2,12401674,21420,['Bill Weyermann Partner AG has been producing...,tf.Tensor(\n[[ 0.05547219 0.04902443 0.01770...,1
3,3,12473085,62100,['Administration Communale de Valderuzforstart...,tf.Tensor(\n[[ 0.01948128 -0.02223398 -0.02341...,0
4,4,12905109,26220;24210,['The Straumann Group is experiencing internat...,tf.Tensor(\n[[-0.02228776 0.03350648 -0.03000...,0
...,...,...,...,...,...,...
195,195,13761500,24210,['SA of the Horlogerie Manufacture Audemars Pi...,tf.Tensor(\n[[ 1.24892173e-02 -2.59787571e-02 ...,0
196,196,13760809,21420,"['20', ' 01', ' 2017Henauer Gugler AGbern80 to...",tf.Tensor(\n[[-0.02621931 -0.07353811 0.06861...,1
197,197,14379272,24210,['Your tasks processing deviations conducting ...,tf.Tensor(\n[[ 0.00078858 0.02764083 0.03774...,0
198,198,14384238,21620,['Text building block header_1 Jobbörse Text m...,tf.Tensor(\n[[ 0.01394392 -0.0571972 0.00221...,1


In [96]:
jobsDf["GREEN_IDENTITY"] = annotatedJobs200["GREEN_IDENTITY"]

In [97]:
jobsDf[jobsDf["GREEN_IDENTITY"].isna()]

,JOB_ID,ISCO,JOB_CONTENT,JOB_SCORES,GREEN_IDENTITY
